In [3]:
import tensorflow as tf

In [4]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train/255.0
x_test = x_test/255.0

In [6]:
x_train[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [7]:
x_train.shape


(60000, 28, 28)

In [8]:
x_image_train = tf.reshape(x_train, [-1, 28, 28, 1])
x_image_train = tf.cast(x_image_train, 'float32')

In [9]:
x_image_train[0]

<tf.Tensor: shape=(28, 28, 1), dtype=float32, numpy=
array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],

In [10]:
x_image_test = tf.reshape(x_test, [-1, 28, 28, 1])

In [11]:
x_image_test = tf.cast(x_image_test, 'float32')

In [45]:
train_ds2 = tf.data.Dataset.from_tensor_slices((x_image_train, y_train)).batch(50)
test_ds2 = tf.data.Dataset.from_tensor_slices((x_image_test, y_test)).batch(50)

In [43]:
y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

InvalidArgumentError: Value for attr 'TI' of float is not in the list of allowed values: uint8, int32, int64
	; NodeDef: {{node OneHot}}; Op<name=OneHot; signature=indices:TI, depth:int32, on_value:T, off_value:T -> output:T; attr=axis:int,default=-1; attr=T:type; attr=TI:type,default=DT_INT64,allowed=[DT_UINT8, DT_INT32, DT_INT64]> [Op:OneHot]

In [14]:
x_image_train = tf.slice(x_image_train, [0,0,0,0], [10000, 28, 28, 1])
y_train = tf.slice(y_train, [0, 0], [10000, 10])

In [15]:
y_train

<tf.Tensor: shape=(10000, 10), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)>

In [16]:
weights_conv1 = tf.Variable(tf.random.truncated_normal([5, 5, 1, 32], stddev = 0.1, seed = 0))
biases_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))

In [17]:
weights_conv1.shape
x_image_train.shape

TensorShape([10000, 28, 28, 1])

In [18]:
def conv1(x):
    return(tf.nn.conv2d(x, weights_conv1, strides=[1,1,1,1], padding='SAME')+ biases_conv1)


In [19]:
def h_conv1(x):
    return tf.nn.relu(conv1(x))

In [20]:
def final_conv1(x):
    return tf.nn.max_pool(h_conv1(x), ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [21]:
weights_conv2 = tf.Variable(tf.random.truncated_normal([5, 5, 32, 64], stddev = 0.1, seed = 1))
biases_conv2 = tf.Variable(tf.constant(0.1, shape = [64]))

In [22]:
def conv2(x):
    return(tf.nn.conv2d(final_conv1(x), weights_conv2, strides=[1,1,1,1], padding='SAME') + biases_conv2)

In [23]:
def h_conv2(x):
    return tf.nn.relu(conv2(x))

In [24]:
def final_conv2(x):
    return tf.nn.max_pool(h_conv2(x), ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [25]:
weights_feature1 = tf.Variable(tf.random.truncated_normal([7*7*64, 1024], stddev=0.1, seed=2))
biases_feature1 = tf.Variable(tf.constant(0.1, shape=[1024]))

In [26]:
def format_layer2(x):
    return tf.reshape(final_conv2(x), [-1, 7*7*64])

In [27]:
def feature1(x):
    return tf.matmul(format_layer2(x), weights_feature1) + biases_feature1

In [28]:
def h_feature1(x):
    return tf.nn.relu(feature1(x))

In [29]:
def drop_layer(x):
    return tf.nn.dropout(h_feature1(x), 0.5)

In [30]:
weights_feature2 = tf.Variable(tf.random.truncated_normal([1024, 10], stddev=0.1, seed=2))
biases_feature2 = tf.Variable(tf.constant(0.1, shape=[10]))

In [31]:
def feature_final(x):
    return tf.matmul(drop_layer(x), weights_feature2) + biases_feature2

In [32]:
def CNN(x):
    return tf.nn.softmax(feature_final(x))

In [49]:
def loss(true, pred):
    
    return (-tf.reduce_sum(true*tf.math.log(pred + 1e-8)))

In [34]:
grad_desc = tf.keras.optimizers.Adam(1e-4)

In [35]:
variables = [weights_conv1, biases_conv1, weights_conv2, biases_conv2,
            weights_feature1, biases_feature1, weights_feature2, biases_feature2]
def train_network(inp_x, exp_y):
    with tf.GradientTape() as tape:
        curr_loss = loss(exp_y, CNN(inp_x))
        grads = tape.gradient(curr_loss, variables)
        grad_desc.apply_gradients(zip(grads, variables))
        return curr_loss.numpy()

In [36]:
is_corr_pred = tf.equal(tf.argmax(CNN(x_image_train), axis=1), tf.argmax(y_train, axis=1))

In [37]:
accuracy = tf.reduce_mean(tf.cast(is_corr_pred, 'float32'))

In [51]:
epochs = 4

for i in range(epochs):
    j = 0
    for x_batch_train, y_batch_train in train_ds2:
        j += 1
        curr_loss = train_network(x_batch_train, y_batch_train)
        if j%50 == 0:
            is_corr_pred = tf.equal(tf.argmax(CNN(x_batch_train), axis=1), tf.argmax(y_batch_train, axis=1))
            accuracy = tf.reduce_mean(tf.cast(is_corr_pred, tf.float32)).numpy()
            
            print(f"epoch: {i} running, batch: {j}, loss: {curr_loss}, accuracy: {accuracy}")
        
    curr_loss = loss(y_train, CNN(x_image_train)).numpy()
    is_corr_pred = tf.equal(tf.argmax(CNN(x_image_train), axis=1), tf.argmax(y_train, axis=1))
    accuracy = tf.reduce_mean(tf.cast(is_corr_pred, tf.float32)).numpy()
    print(f"epoch {i} ended, loss: {curr_loss}, accuracy: {accuracy}")

epoch: 0 running, batch: 50, loss: 2.9499690532684326, accuracy: 0.9800000190734863
epoch: 0 running, batch: 100, loss: 14.087347984313965, accuracy: 0.9399999976158142
epoch: 0 running, batch: 150, loss: 3.877556085586548, accuracy: 0.9599999785423279
epoch: 0 running, batch: 200, loss: 2.776298999786377, accuracy: 1.0
epoch 0 ended, loss: 1413.77587890625, accuracy: 0.95660001039505
epoch: 1 running, batch: 50, loss: 4.107239246368408, accuracy: 1.0
epoch: 1 running, batch: 100, loss: 9.501497268676758, accuracy: 0.9399999976158142
epoch: 1 running, batch: 150, loss: 4.822587013244629, accuracy: 0.9800000190734863
epoch: 1 running, batch: 200, loss: 1.8242661952972412, accuracy: 1.0
epoch 1 ended, loss: 1178.4134521484375, accuracy: 0.9610000252723694
epoch: 2 running, batch: 50, loss: 1.8473315238952637, accuracy: 0.9599999785423279
epoch: 2 running, batch: 100, loss: 6.753937721252441, accuracy: 0.9399999976158142
epoch: 2 running, batch: 150, loss: 6.0916290283203125, accuracy: 0.

In [46]:
y_train.shape


TensorShape([10000, 10])